# 프롬프트 엔지니어링

## 환경변수 로딩

In [1]:
# API 키로딩
from dotenv import load_dotenv
import os

load_dotenv('.env', override=True)
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

## 출력의 예시를 제공하는 형태
- zero-shot, one-shot, few-shot

### zero-shot

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-3.5-turbo")

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 문장의 감정을 분류하는 AI다."),
    ("human", "문장: {text}\n감정:")
])

chain = prompt | llm

response = chain.invoke({"text": "이 서비스는 정말 최악이야"})
print(response.content)

부정적인 감정


### one-shot

In [3]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 문장의 감정을 긍정 또는 부정으로 분류하는 AI다."),
    ("human", """
문장: 이 영화는 정말 재미있었다
감정: 긍정

문장: {text}
감정:
""")
])

chain = prompt | llm

response = chain.invoke({"text": "고객센터 응대가 너무 불친절해"})
print(response.content)

부정


### few-shot

In [5]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate
)

examples = [
    {"input": "이 영화는 정말 재미있었다", "output": "긍정"},
    {"input": "서비스 품질이 형편없다", "output": "부정"},
    {"input": "가격 대비 만족스러웠다", "output": "긍정"},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "문장: {input}"),
    ("ai", "감정: {output}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 문장의 감정을 긍정 또는 부정으로 분류하는 AI다."),
    few_shot_prompt,
    ("human", "문장: {text}")
])

chain = final_prompt | llm

response = chain.invoke({"text": "배송이 너무 느리고 불편했다"})
print(response.content)

감정: 부정


## 사고의 단계
- 개념: 모델에게 "단계별로 생각해보자"고 요청하여 논리적 비약을 막고 정확도를 높이는 기법

In [6]:
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# llm = ChatOpenAI(model="gpt-4o")
llm = init_chat_model(
    api_key=GEMINI_API_KEY,
    model="models/gemini-2.5-flash",   
    model_provider="google-genai",
    temperature=0.5 
    )

# [실습] Zero-shot CoT: 마법의 구문 "Step-by-step"
cot_prompt = ChatPromptTemplate.from_template(
    "우리 회사의 지난분기 이탈 고객이 15% 증가했습니다. 원인을 분석하고 대응 전략을 '단계별로' 세워주세요."
)

chain = cot_prompt | llm | StrOutputParser()

result = chain.invoke({})

In [7]:
from IPython.display import display, Markdown
print("--- [Zero-shot CoT 결과] ---")
# 마크다운으로 렌더링하여 출력
display(Markdown(result))

--- [Zero-shot CoT 결과] ---


지난 분기 이탈 고객 15% 증가는 매우 심각한 상황이며, 즉각적이고 체계적인 대응이 필요합니다. 단순히 숫자가 늘어난 것을 넘어, 근본적인 원인을 파악하고 장기적인 관점에서 회사의 경쟁력을 강화하는 기회로 삼아야 합니다.

---

## 1단계: 원인 분석 (Diagnostic Phase)

가장 먼저 해야 할 일은 '왜' 이탈률이 증가했는지 파악하는 것입니다. 가설을 세우고 데이터를 통해 검증하는 과정이 필수적입니다.

### 1.1 데이터 기반 분석

*   **이탈 고객 특성 분석:**
    *   **어떤 고객층에서 이탈이 증가했나?** (신규 고객, 장기 고객, 특정 세그먼트, 특정 서비스 이용 고객 등)
    *   **이탈 전 고객 행동은 어떠했나?** (서비스 사용 빈도 감소, 특정 기능 사용 중단, 고객 지원 문의 증가 등)
    *   **이탈 시점은 언제였나?** (서비스 가입 후 특정 기간 내, 특정 이벤트/캠페인 이후 등)
    *   **결제 방식/플랜별 이탈률 변화는?** (무료 플랜에서 유료 전환 실패, 특정 유료 플랜 고객 이탈 등)
*   **고객 여정(Customer Journey) 분석:**
    *   **어떤 단계에서 이탈이 많이 발생했나?** (온보딩, 특정 기능 사용, 재구매/재계약 등)
    *   **고객이 경험하는 문제점은 없었나?** (로그 분석, 사용자 행동 패턴 분석)
*   **VOC(Voice of Customer) 분석:**
    *   **고객센터 문의 내용:** 이탈 전 문의 내용 중 불만사항, 해결되지 않은 이슈 파악.
    *   **취소/해지 사유:** 해지 신청 시 남긴 사유를 면밀히 분석 (설문, 직접 문의 내용).
    *   **온라인 리뷰/피드백:** 외부 채널(SNS, 커뮤니티, 앱 스토어 등)의 부정적인 피드백 추이 분석.

### 1.2 잠재적 원인 가설 수립 및 검증

위 데이터 분석을 토대로 아래와 같은 잠재적 원인 가설을 세우고, 관련 데이터를 찾아 검증합니다.

*   **[제품/서비스 품질 문제]**
    *   **가설:** 최근 서비스 오류, 성능 저하, 기능 부족, UX 불편 등 제품 자체의 문제로 고객 만족도가 하락했다.
    *   **검증:** 시스템 장애 기록, 버그 리포트, 사용자 만족도 조사(NPS, CSAT) 변화, 특정 기능 사용률/이탈률, 경쟁사 대비 기능/성능 비교.
*   **[가격/가치 불균형]**
    *   **가설:** 고객이 느끼는 서비스 가치 대비 가격이 비싸다고 느끼거나, 경쟁사 대비 가성비가 떨어진다고 판단했다.
    *   **검증:** 경쟁사 가격 및 제공 기능 비교, 요금제 변경 이력, 고객 피드백 내 가격 언급 빈도.
*   **[고객 지원/경험 문제]**
    *   **가설:** 고객센터 응대 지연, 문제 해결 미흡, 불친절 등 고객 지원 품질이 저하되어 불만이 쌓였다.
    *   **검증:** 고객센터 응대 시간, 문제 해결률, 상담원 만족도, VOC 분석 (불만 유형).
*   **[경쟁사 요인]**
    *   **가설:** 경쟁사가 더 나은 제품/서비스, 파격적인 프로모션으로 고객을 유인하고 있다.
    *   **검증:** 경쟁사 신규 서비스 출시, 대규모 마케팅 캠페인, 가격 정책 변화, 시장 점유율 변화.
*   **[마케팅/온보딩 문제]**
    *   **가설:** 신규 고객 유입 시 서비스에 대한 기대치를 과도하게 높였거나, 온보딩 과정이 미흡하여 초기 이탈이 늘었다.
    *   **검증:** 신규 가입 고객의 초기 사용률, 온보딩 완료율, 가입 채널별 이탈률.
*   **[시장 환경 변화]**
    *   **가설:** 경기 침체, 산업 트렌드 변화 등으로 인해 고객의 니즈가 변화했거나, 서비스 이용 예산이 줄었다.
    *   **검증:** 거시 경제 지표, 관련 산업 보고서, 고객 설문조사(니즈 변화).

---

## 2단계: 대응 전략 수립 및 실행 (Action Phase)

원인 분석을 통해 핵심 문제가 파악되면, 이를 해결하기 위한 단계별 전략을 수립하고 실행해야 합니다.

### 단계 1: 긴급 조치 및 단기 대응 (Immediate / Short-term)

**목표:** 더 이상의 이탈 확산을 막고, 이탈 위험 고객을 선제적으로 관리하며, 핵심 문제를 빠르게 해결합니다.

1.  **위험 고객 식별 및 선제적 관리:**
    *   **이탈 예측 모델 활용:** 이탈 징후가 보이는 고객(사용량 감소, 특정 기능 미사용, 문의 증가 등)을 식별합니다.
    *   **개인화된 리텐션 캠페인:** 식별된 고객에게 맞춤형 혜택(할인, 추가 기능 제공, 무료 컨설팅 등)과 함께 서비스의 가치를 재강조하는 메시지를 전달합니다.
    *   **담당자 지정 및 직접 소통:** VIP 고객이나 이탈 위험도가 높은 핵심 고객에게는 전담 인력을 배정하여 직접 소통하고 문제를 파악/해결합니다.
2.  **핵심 이탈 원인 긴급 개선:**
    *   **치명적인 버그/성능 문제 해결:** 만약 제품/서비스 품질 문제가 주원인이라면, 가장 시급한 버그나 성능 저하 요인을 최우선으로 개선하고 즉시 고객에게 공지합니다.
    *   **고객 지원 역량 강화:** CS 응대 지연이 문제라면, 인력 확충, 교육 강화, 챗봇/FAQ 시스템 개선 등을 통해 빠른 응대 및 문제 해결 능력을 향상시킵니다.
    *   **이탈 고객 대상 Win-back 캠페인:** 최근 이탈한 고객들에게 개인화된 제안(재가입 할인, 특정 기능 무료 이용 등)과 함께 이탈 원인 개선 내용을 알리며 재유입을 유도합니다.

### 단계 2: 중기적 개선 및 프로세스 최적화 (Medium-term)

**목표:** 이탈 원인의 근본적인 해결책을 마련하고, 고객 만족도 및 충성도를 높이는 지속 가능한 시스템을 구축합니다.

1.  **제품/서비스 개선 로드맵 재수정:**
    *   **고객 피드백 반영:** 이탈 원인 분석 결과와 VOC를 제품 개발 로드맵에 적극 반영하여 고객이 진짜 필요로 하는 기능을 개발하거나 기존 기능을 개선합니다.
    *   **경쟁 우위 확보:** 경쟁사 대비 약점이었던 부분을 보완하고, 차별화된 가치를 제공할 수 있는 기능을 기획/개발합니다.
    *   **온보딩 경험 개선:** 신규 고객이 서비스의 가치를 빠르게 인지하고 활용할 수 있도록 온보딩 과정을 직관적이고 효과적으로 개선합니다(튜토리얼, 가이드, 초기 세팅 지원).
2.  **가격 및 가치 제안 재검토:**
    *   **고객 체감 가치 증대:** 서비스가 제공하는 핵심 가치를 명확히 커뮤니케이션하고, 고객이 실제 얻는 이점을 강조합니다.
    *   **요금제 구조 최적화:** 시장 상황, 경쟁사 가격, 고객 세그먼트별 니즈를 고려하여 요금제 구조를 재편하거나, 부가 서비스 옵션을 다양화하여 선택의 폭을 넓힙니다.
3.  **고객 커뮤니케이션 강화:**
    *   **정기적인 가치 전달:** 이메일, 앱 푸시, 인앱 메시지 등을 통해 새로운 기능, 팁, 성공 사례 등을 정기적으로 공유하여 고객이 서비스의 가치를 지속적으로 인지하도록 합니다.
    *   **양방향 소통 채널 활성화:** 고객 설문, 포럼, 사용자 그룹 등을 통해 고객의 의견을 꾸준히 듣고, 반영 과정을 투명하게 공개하여 신뢰를 구축합니다.

### 단계 3: 장기적 관점의 예방 및 성장 전략 (Long-term / Strategic)

**목표:** 이탈을 사전에 방지하고, 고객 충성도를 극대화하여 지속적인 성장을 이끌어냅니다.

1.  **예측 분석 시스템 고도화:**
    *   **고도화된 이탈 예측 모델:** 머신러닝 기반의 이탈 예측 모델을 구축하여, 이탈 징후를 보다 정확하고 빠르게 감지하고 선제적인 대응이 가능하도록 합니다.
    *   **자동화된 대응 시스템:** 이탈 징후 발생 시, 자동화된 메시지 발송, 혜택 제안 등의 액션을 통해 리텐션 노력을 효율화합니다.
2.  **고객 충성도 프로그램 운영:**
    *   **로열티 프로그램 도입:** 장기 고객에게 특별한 혜택(포인트, 등급별 할인, 전용 지원 등)을 제공하여 충성도를 높입니다.
    *   **커뮤니티 구축:** 고객 간의 교류를 장려하고, 서비스 관련 지식을 공유하는 커뮤니티를 구축하여 고객이 서비스에 소속감과 유대감을 느끼도록 합니다.
3.  **시장 및 경쟁 환경 지속 모니터링:**
    *   **경쟁사 동향 분석:** 경쟁사의 신규 서비스, 가격 정책, 마케팅 전략 등을 지속적으로 모니터링하여 우리의 전략에 반영합니다.
    *   **시장 트렌드 예측:** 산업 전반의 변화, 기술 트렌드, 고객 니즈 변화 등을 예측하여 서비스 개발 및 비즈니스 전략에 반영함으로써 미래의 이탈 요인을 사전에 차단합니다.
4.  **피드백 루프 구축 및 문화 내재화:**
    *   **전사적 고객 중심 문화:** 모든 부서가 고객 피드백을 중요하게 여기고, 이탈률 관리를 전사적인 목표로 설정하며, 정기적으로 이탈 원인 분석 및 개선 방안을 논의하는 문화를 정착시킵니다.
    *   **지속적인 측정 및 학습:** 이탈률, NPS, CSAT 등 핵심 지표를 꾸준히 측정하고, 각 단계의 전략 실행 결과를 분석하여 다음 전략에 반영하는 학습 사이클을 구축합니다.

---

**가장 중요한 점:** 이 모든 과정은 **데이터 기반**으로 이루어져야 하며, **각 부서 간의 긴밀한 협업**이 필수적입니다. 이탈률 15% 증가는 위기이지만, 이를 통해 고객 경험을 깊이 있게 이해하고 회사의 서비스와 프로세스를 한 단계 업그레이드할 수 있는 중요한 기회가 될 수 있습니다.

## 신뢰의 기술 (Self-Consistency)
- 개념: 동일한 질문을 여러 번 던져 가장 많이 나오는 결론을 선택함으로써 답변의 일관성을 확보

In [8]:
from langchain_core.runnables import RunnableParallel

# 온도를 높여 다양한 사고 경로 유도
llm_creative = ChatOpenAI(model="gpt-4o", temperature=0.7)

consistency_prompt = ChatPromptTemplate.from_template(
    "A사에서 생성형 AI를 도입할 때, 자체 서버(On-premise)와 클라우드 중 어느 것이 장기적으로 유리할까요?" 
)

# 3개의 독립적인 추론 경로를 병렬로 실행
consistency_chain = RunnableParallel(
    path1=consistency_prompt | llm_creative | StrOutputParser(),
    path2=consistency_prompt | llm_creative | StrOutputParser(),
    path3=consistency_prompt | llm_creative | StrOutputParser()
)

print("--- [Self-Consistency 실행] ---")
result = consistency_chain.invoke({})

--- [Self-Consistency 실행] ---


In [9]:
print(f"결과 1: {result['path1'][:100]}...")
print(f"결과 2: {result['path2'][:100]}...")
print(f"결과 3: {result['path3'][:100]}...")

결과 1: 생성형 AI를 도입할 때 자체 서버(On-premise)와 클라우드 중 어느 것이 장기적으로 유리한지는 여러 요인에 따라 달라질 수 있습니다. 다음은 두 가지 옵션의 장단점을 고려...
결과 2: 생성형 AI를 도입할 때 자체 서버(On-premise)와 클라우드 중 어느 것이 장기적으로 유리한지는 여러 요인에 따라 다릅니다. 각각의 옵션에는 장단점이 있으며, A사의 특정 ...
결과 3: A사에서 생성형 AI를 도입할 때, 자체 서버(On-premise)와 클라우드 중 어떤 것이 장기적으로 유리한지는 여러 요인에 따라 달라질 수 있습니다. 다음은 각각의 장단점입니다...


In [10]:
print(result['path1'])

생성형 AI를 도입할 때 자체 서버(On-premise)와 클라우드 중 어느 것이 장기적으로 유리한지는 여러 요인에 따라 달라질 수 있습니다. 다음은 두 가지 옵션의 장단점을 고려하여 A사가 장기적으로 어떤 선택을 할 수 있을지에 대한 분석입니다.

### 자체 서버(On-premise)의 장점:
1. **데이터 보안 및 프라이버시**: 민감한 데이터를 처리하는 경우, 데이터가 외부 서버로 이동하지 않기 때문에 보안과 프라이버시를 더 잘 관리할 수 있습니다.
2. **커스터마이제이션**: 시스템을 완전히 제어할 수 있어 필요에 따라 소프트웨어 및 하드웨어를 맞춤 설정할 수 있습니다.
3. **고정 비용**: 초기 투자 비용이 크지만, 장기적으로 봤을 때 운영 비용이 예측 가능하며 고정적일 수 있습니다.

### 자체 서버(On-premise)의 단점:
1. **초기 비용**: 서버 및 관련 인프라 구축에 대한 초기 비용이 매우 높습니다.
2. **유지보수**: 하드웨어 및 소프트웨어의 유지보수, 업데이트, 확장에 대한 책임이 내부에 있습니다.
3. **확장성 부족**: 필요에 따라 시스템을 확장하는 것이 클라우드에 비해 어렵고 시간이 걸릴 수 있습니다.

### 클라우드의 장점:
1. **확장성**: 필요에 따라 쉽게 리소스를 확장하거나 축소할 수 있어 유연성이 뛰어납니다.
2. **비용 효율성**: 초기 투자 비용이 낮고, 사용한 만큼 비용을 지불하는 모델이므로 초기 부담이 적습니다.
3. **최신 기술 활용**: 클라우드 서비스 제공업체가 지속적으로 기술을 업데이트하므로 최신 기술을 쉽게 활용할 수 있습니다.

### 클라우드의 단점:
1. **데이터 보안 우려**: 데이터가 외부 서버에 저장되기 때문에 보안 및 프라이버시 문제가 발생할 수 있습니다.
2. **종속성**: 특정 클라우드 벤더에 종속될 가능성이 있으며, 벤더의 정책 및 가격 변동에 영향을 받을 수 있습니다.
3. **지연 시간**: 네트워크를 통해 데이터 전송이 이루어지기 때문에 지연 시간이

In [11]:
print(result['path2'])

생성형 AI를 도입할 때 자체 서버(On-premise)와 클라우드 중 어느 것이 장기적으로 유리한지는 여러 요인에 따라 다릅니다. 각각의 옵션에는 장단점이 있으며, A사의 특정 요구사항과 상황에 따라 최선의 선택이 달라질 수 있습니다. 다음은 두 가지 옵션을 비교한 것입니다.

### 자체 서버(On-premise)
#### 장점:
1. **데이터 보안**: 민감한 데이터를 회사 내부에 보관하므로 보안에 대한 통제력이 높습니다.
2. **맞춤형 환경**: 특정 요구에 맞게 시스템을 최적화할 수 있는 자유도가 높습니다.
3. **지속적인 비용**: 초기 구축 비용이 높지만, 장기적으로 클라우드에 비해 고정 비용이 될 수 있습니다.

#### 단점:
1. **초기 비용**: 하드웨어 및 인프라 구축에 대한 초기 투자 비용이 큽니다.
2. **유지보수**: 시스템의 유지보수와 업그레이드에 대한 책임이 내부에 있습니다.
3. **확장성**: 수요 변화에 신속하게 대응하기 어렵습니다.

### 클라우드
#### 장점:
1. **확장성**: 필요에 따라 자원을 쉽게 확장하거나 축소할 수 있습니다.
2. **초기 비용 절감**: 초기 하드웨어 투자 없이 사용한 만큼만 비용을 지불합니다.
3. **최신 기술 접근**: 클라우드 제공업체의 최신 기술과 업데이트를 쉽게 이용할 수 있습니다.

#### 단점:
1. **데이터 보안**: 데이터가 외부 서버에 저장되므로 보안 우려가 있을 수 있습니다.
2. **의존성**: 특정 클라우드 제공업체에 대한 의존성이 발생할 수 있습니다.
3. **장기 비용**: 장기적으로 볼 때, 사용량이 많아지면 비용이 증가할 수 있습니다.

### 결론
A사가 어떤 선택을 할지는 데이터 보안 요구사항, 예산, 기술 역량, 확장성 필요성 등 여러 요인을 고려해야 합니다. 만약 데이터 보안이 최우선이라면 자체 서버가 유리할 수 있지만, 유연성과 초기 비용 면에서 클라우드가 더 적합할 수 있습니다. 이외에도 하이브리드 접근 방식(자체 서버와 클라우드를 

## 추론과 행동 (ReAct 기초)
- 개념: AI가 스스로 생각(Thought)하고, 행동(Action)을 결정하는 자율적 흐름을 구축함.

In [12]:
react_system = """너는 전략적 AI 에이전트다. 질문에 대해 다음 형식을 엄격히 지켜라.
1. Thought: 현재 질문에 대해 분석한다.
2. Action: '직접 답변' 또는 '외부 검색 필요' 중 하나를 선택한다.
3. Response: 최종 답변을 작성한다.

질문: {input}"""

react_prompt = ChatPromptTemplate.from_template(react_system)
react_chain = react_prompt | llm | StrOutputParser()

In [13]:
result = react_chain.invoke({"input": "현재 엔비디아(NVIDIA)의 주가 수익비율(PER)을 바탕으로 투자 가치를 분석해줘."})

In [14]:
print("--- [ReAct 사고 과정 확인] ---")
print(result)

--- [ReAct 사고 과정 확인] ---
1. Thought: 사용자는 엔비디아(NVIDIA)의 '현재' 주가 수익비율(PER)을 바탕으로 투자 가치를 분석해달라고 요청했습니다. '현재' PER은 실시간으로 변동하는 금융 데이터이므로, 가장 정확한 분석을 위해서는 실시간 데이터 조회가 필수적입니다. 따라서 외부 검색이 필요하며, 검색 후 해당 데이터를 바탕으로 분석 프레임워크를 적용하여 투자 가치를 평가해야 합니다. PER 분석 시에는 단순 수치뿐 아니라 산업 평균, 성장률, 미래 전망 등을 종합적으로 고려해야 합니다.

2. Action: 외부 검색 필요

3. Response:
현재 엔비디아(NVIDIA)의 실시간 주가 수익비율(PER)은 시장 상황과 주가 변동에 따라 실시간으로 변동합니다. 따라서 정확한 '현재' PER 수치를 바탕으로 한 투자 가치 분석을 위해서는 실시간 금융 데이터를 조회해야 합니다.

**[외부 검색 후 분석에 필요한 정보 및 분석 방법]**

외부 검색을 통해 엔비디아의 최신 PER 정보를 얻었다면, 다음 요소들을 고려하여 투자 가치를 분석할 수 있습니다.

1.  **현재 PER 수치 확인:**
    *   가장 최근의 TTM(Trailing Twelve Months, 지난 12개월 실적 기준) PER과 FWD(Forward, 향후 12개월 예상 실적 기준) PER을 확인합니다. FWD PER은 미래 성장 기대치를 반영합니다.

2.  **산업 및 경쟁사 비교:**
    *   엔비디아가 속한 반도체 산업(특히 AI/데이터센터 GPU, 게이밍 GPU 분야)의 평균 PER과 AMD, Intel, Broadcom 등 주요 경쟁사들의 PER과 비교합니다.
    *   엔비디아의 PER이 산업 평균이나 경쟁사 대비 현저히 높다면, 시장이 엔비디아의 미래 성장성에 매우 높은 기대를 걸고 있다는 의미이며, 이는 그만큼 높은 성장률이 뒷받침되어야 정당화될 수 있습니다.

3.  **역사적 PER 추이 비교:**
    *   엔비디아

## 비즈니스 로직 통합 설계
- DX 컨설턴트가 되어 '전통 시장 디지털 전환 솔루션'을 설계하는 통합 체인을 만들기

In [15]:
# 1. 현황 분석 체인
analysis_step = ChatPromptTemplate.from_template("{market_name}의 디지털화 저해 요인을 분석해줘.")

# 2. 솔루션 도출 체인
solution_step = ChatPromptTemplate.from_template("다음 분석을 바탕으로 소상공인을 위한 AX(AI 전환) 로드맵을 작성해줘: {analysis}")

# 통합 비즈니스 로직
ax_consulting_chain = (
    {"analysis": analysis_step | llm | StrOutputParser()}
    | solution_step
    | llm
    | StrOutputParser()
)

In [16]:
print("--- [DX 컨설팅 통합 리포트] ---")
print(ax_consulting_chain.invoke({"market_name": "남대문 시장"}))

--- [DX 컨설팅 통합 리포트] ---
## 남대문 시장 소상공인을 위한 AX(AI 전환) 로드맵

남대문 시장의 디지털화 저해 요인 분석을 바탕으로, 상인들의 디지털 문해력과 변화 저항을 고려하면서도 AI 기술의 잠재력을 최대한 활용할 수 있는 단계별 AX(AI 전환) 로드맵을 제안합니다. 이 로드맵은 단순히 기술 도입을 넘어, 상인들의 인식 변화와 시장의 문화적 특성을 존중하며 점진적으로 AI 기반 혁신을 이루는 데 초점을 맞춥니다.

---

### 로드맵 비전:
**"전통의 가치를 AI와 함께, 스마트한 남대문 시장으로 도약하여 글로벌 고객을 사로잡는 활력 넘치는 명품 시장 구축"**

### 로드맵 목표:
1.  **AI 기반 운영 효율화:** 재고, 주문, 고객 관리 등 핵심 업무 프로세스에 AI를 도입하여 상인들의 업무 부담을 경감하고 효율성을 극대화합니다.
2.  **데이터 기반 맞춤형 고객 경험 제공:** 고객 데이터를 활용한 AI 추천 시스템, 개인화된 마케팅으로 고객 만족도를 높이고 재방문을 유도합니다.
3.  **새로운 성장 동력 확보:** 온라인 판매 채널 확장, 글로벌 고객 유치, 데이터 기반 신상품 개발 등 AI를 통한 비즈니스 모델 혁신을 추구합니다.
4.  **디지털 격차 해소 및 상생:** 상인들의 디지털 역량을 강화하고, AI 전환 과정에서 상인 간 협력과 시장 전체의 시너지를 창출합니다.

---

### 남대문 시장 AX(AI 전환) 로드맵 단계

**Phase 0: AX 전환 준비 및 인식 전환 (기간: 3~6개월) - 'AI의 씨앗 뿌리기'**

*   **목표:** AI 전환의 필요성을 공감하고, 기초적인 디지털 문해력을 확보하며, 변화에 대한 긍정적인 분위기 조성
*   **주요 저해 요인 해소:** 인적/문화적 요인 (고령화, 저항감, 인식 부족), 정부/지자체 지원의 한계 (맞춤형 지원 필요)

1.  **AI 전환 비전 공유 및 공감대 형성:**
    *   **활동:** 'AI가 남대문 시장에 가져올 미

## 모델별 추론 비교(GPT vs Gemini)

In [17]:
from langchain.chat_models import init_chat_model

gpt = init_chat_model(
    api_key=OPENAI_API_KEY, 
    model="gpt-4o-mini",
    model_provider="openai",
    temperature=0.5
    )
gemini = init_chat_model(
    api_key=GEMINI_API_KEY,
    model="models/gemini-2.5-flash",
    model_provider="google-genai",
    temperature=0.5
    )

def run_comparison(topic, model):
    market_name = {"market_name": topic}
    # GPT-4o와 Gemini 1.5를 사용한 결과 비교
    model_chain = {"analysis": analysis_step | model | StrOutputParser()} | solution_step | model | StrOutputParser()
    model_result = model_chain.invoke(market_name)

    return model_result

In [18]:
# gpt 모델 추론
gpt_result = run_comparison("오프라인 중고 서점", gpt)

In [19]:

# gemini 모델 추론
gemini_result = run_comparison("오프라인 중고 서점", gemini)

In [20]:
# 결과 출력
print(f"## GPT-4o 결과:\n{gpt_result}\n")
print(f"## Gemini 1.5-flash 결과:\n{gemini_result}")

## GPT-4o 결과:
소상공인을 위한 AX(AI 전환) 로드맵은 중고 서점이 디지털화의 도전 과제를 극복하고, 효율성을 높이며, 고객 경험을 개선할 수 있도록 돕는 데 중점을 두어야 합니다. 다음은 이러한 목표를 달성하기 위한 단계별 로드맵입니다.

### AX 로드맵

#### 1단계: 기초 기술 인프라 구축
- **기술 인프라 투자**: 저렴한 비용으로 웹사이트 및 모바일 애플리케이션 구축. 클라우드 기반 솔루션을 활용하여 초기 비용 절감.
- **디지털 교육 프로그램**: 직원들을 대상으로 기본적인 디지털 기술 교육 및 워크숍을 실시하여 디지털 역량 강화.
- **기초 데이터 관리 시스템 도입**: 재고 관리 및 판매 데이터를 수집하고 분석할 수 있는 간단한 시스템 구축.

#### 2단계: 온라인 판매 플랫폼 개발
- **온라인 쇼핑몰 구축**: 사용자가 편리하게 중고 서적을 검색하고 구매할 수 있는 웹사이트 및 모바일 앱 개발.
- **결제 시스템 도입**: 안전하고 간편한 온라인 결제 시스템을 구축하여 고객의 구매 경험 향상.
- **물류 및 배송 시스템 통합**: 효율적인 물류 관리 시스템을 도입하여 배송 과정의 효율성을 높임.

#### 3단계: 고객 경험 향상
- **고객 피드백 시스템 구축**: 고객의 의견을 수집하고 분석하여 서비스 개선에 반영. 이를 통해 고객 충성도 향상.
- **개인화된 추천 시스템**: AI 기술을 활용하여 고객의 구매 이력 및 선호도를 분석하고, 맞춤형 추천 제공.
- **가상 서점 체험**: AR/VR 기술을 활용하여 고객이 온라인에서 책을 직접 보고 선택할 수 있는 경험 제공.

#### 4단계: 마케팅 및 경쟁력 강화
- **디지털 마케팅 전략 수립**: SNS, 이메일 마케팅, 콘텐츠 마케팅 등 다양한 채널을 통해 브랜드 인지도 및 고객 유치.
- **고객 커뮤니티 형성**: 온라인 커뮤니티를 통해 고객과의 관계를 강화하고, 지역 사회와의 연결을 유지.
- **가격 경쟁력 분석**: 경쟁사 분석을 통해 가